In [17]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

In [8]:
df = pd.read_csv("C:\\Users\\cflin\\OneDrive\\Desktop\\ANA 680\\Week 1\\Churn_Modelling.csv")

df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
#Splitting Target Feature

y = df["Exited"].values

X = df.drop(columns=["RowNumber", "CustomerId", "Surname", "Exited"])


In [12]:
#Test/train, encoding, and scailing

cat_cols = ["Geography", "Gender"]
num_cols = [c for c in X.columns if c not in cat_cols]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False), cat_cols)
    ],
    remainder="passthrough"
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

X_train_enc = preprocessor.fit_transform(X_train)
X_test_enc  = preprocessor.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_enc)
X_test_scaled  = scaler.transform(X_test_enc)


In [14]:
#Ranking features

ohe = preprocessor.named_transformers_["cat"]

try:
    cat_feature_names = ohe.get_feature_names_out(cat_cols)
except AttributeError:
    cat_feature_names = ohe.get_feature_names(cat_cols)

feature_names = list(cat_feature_names) + num_cols

rf = RandomForestClassifier(n_estimators=300, random_state=42)
rf.fit(X_train_enc, y_train)

importances = rf.feature_importances_
top_idx = np.argsort(importances)[::-1][:10]

print("Top Features:")
for i in top_idx:
    print(f"{feature_names[i]:<25}  importance={importances[i]:.4f}")


Top Features:
Age                        importance=0.2382
EstimatedSalary            importance=0.1461
CreditScore                importance=0.1436
Balance                    importance=0.1431
NumOfProducts              importance=0.1288
Tenure                     importance=0.0806
IsActiveMember             importance=0.0401
Geography_Germany          importance=0.0285
HasCrCard                  importance=0.0190
Gender_Male                importance=0.0179


In [18]:
#Three layer ANN

ann = Sequential([
    Input(shape=(X_train_scaled.shape[1],)),
    Dense(units=6, activation="relu"),
    Dense(units=6, activation="relu"),
    Dense(units=1, activation="sigmoid")
])

ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [24]:
#Train

ann.fit(X_train_scaled, y_train, batch_size=32, epochs=50, verbose=1)

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8596 - loss: 0.3316 
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8606 - loss: 0.3311 
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8609 - loss: 0.3311 
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8604 - loss: 0.3314 
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8614 - loss: 0.3310 
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8612 - loss: 0.3309 
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8618 - loss: 0.3310 
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8615 - loss: 0.3309 
Epoch 9/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8611 - loss: 0.3310 
Epoch 10/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8618 - loss: 0.3309 
Epoch 11/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8614 - loss: 0.3310 
Epoch 12/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 

In [23]:
#Eval

y_prob = ann.predict(X_test_scaled)
y_pred = (y_prob > 0.5).astype(int).ravel()

cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

print("Confusion Matrix:\n", cm)
print("Accuracy:", acc)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Confusion Matrix:
 [[1530   63]
 [ 214  193]]
Accuracy: 0.8615
